## STEP 1: 套件匯入

In [1]:
import requests # HTTP 請求套件
import pandas as pd # 資料清整與匯出套件
from lxml import etree # XPATH 定位套件

## STEP 2: 請求 PTT Law 版

嘗試請求 PPT Law 版，確認響應內容為爬蟲目標內容後，將 HTML 程式碼代入並初始化定位工具。

In [2]:
## PTT Law 版連接
link = "https://www.ptt.cc/bbs/LAW/index.html"

## 請求 PTT Law 版第 1 頁
res = requests.get(link)

## 查看請求的狀態碼
print(res.status_code)

200


In [15]:
## 初始化 XPATH 定位工具的套件
raw_data = etree.HTML(res.text)

## STEP 3: 利用 XPATH 語言解析 HTML 

本次任務僅需爬取 PPT Law 版的第 1 頁與每個討論項目的留言，因此在嘗試請求 PPT Law 版成功後，可以發現請求內容與爬蟲內容相符，但每項討論的留言都埋在個別討論分頁內，因此接下來實作流程將拆成兩個部分：

* 第 1 部分：在 PTT Law 版爬取每個討論項目連接

* 第 2 部分：藉由爬取每個討論項目的連接，迴圈個別進行主題、發問者、日期、留言資訊等爬蟲

In [17]:
# 第 1 部分
## 了解 PTT Law 版第 1 頁的總討論項目數量
total_posts = len(raw_data.xpath("//div[@class='title']"))
## 藉由建立 links 的 List，將爬取的個別討論項目連接保存
links = []

## 利用 Range 進行迴圈，把個別討論連接保存到 links 參數中
for no_post in range(1,total_posts+1):
    ## link 為個別討論項目的連接，因此每輪的迴圈將覆蓋上一次迴圈的值
    link = raw_data.xpath(f"(//div[@class='title']//a)[{no_post}]/@href")[0]
    print("Current Link: ",link)
    links.append(link)

Current Link:  /bbs/LAW/M.1639278563.A.F42.html
Current Link:  /bbs/LAW/M.1639292679.A.A1C.html
Current Link:  /bbs/LAW/M.1639313357.A.B30.html
Current Link:  /bbs/LAW/M.1639355860.A.0F9.html
Current Link:  /bbs/LAW/M.1639384334.A.868.html
Current Link:  /bbs/LAW/M.1639386649.A.8CD.html
Current Link:  /bbs/LAW/M.1639461127.A.C01.html
Current Link:  /bbs/LAW/M.1139760328.A.8DB.html
Current Link:  /bbs/LAW/M.1629653620.A.1E6.html
Current Link:  /bbs/LAW/M.1629654896.A.827.html
Current Link:  /bbs/LAW/M.1629656635.A.206.html
Current Link:  /bbs/LAW/M.1635743148.A.0A7.html


In [24]:
# 第 2 部分
## 在爬完個別討論項目後會發現，連接不是完整的，因此宣告 base_link 參數來補上前段
base_link = "https://www.ptt.cc"
## 在進行個別項目分頁的爬蟲時，將爬到的所有資訊以 List of Dict 的形式儲存至 data_to_df 參數中
data_to_df = []

## 由於用 1 個迴圈無法爬取所有留言，因此使用兩個迴圈（別名：巢狀迴圈）的方式進行
for link in links:
    ## 針對個別討論項目進行分頁的請求
    res_each = requests.get(base_link+link)
    ## 將響應的結果代入 XPATH 定位工具並初始化
    raw_data_each = etree.HTML(res_each.text)
    
    ## 為了爬取個別討論項目分頁中的所有留言，因此需要透過迴圈完成，在這之前計算該分頁的總留言數
    ## 在撰寫程式中，可以使用 len 來為 raw_data_each.xpath("(//span[@class='f3 hl push-userid'])")
    ## 返回的 list 進行長度計算，即 list 的長度等於總留言數
    total_comment = len(raw_data_each.xpath("(//span[@class='f3 hl push-userid'])"))
    ## 用於儲存該輪討論項目中的所有留言，將爬到的所有留言以 List of Dict 的形式儲存至 comments 參數中
    comments = []
    
    ## 利用計算出的總留言數進行迴圈
    for no_comment in range(1,total_comment+1):
        ## 每輪迴圈即為單項留言，並以迴圈的方式完成所有留言爬取
        ## 將單項留言中的用戶、留言內容與留言時間保存到 comments 參數中
        comments.append({
            "author" : raw_data_each.xpath(f"(//span[@class='f3 hl push-userid'])[{no_comment}]/text()")[0],
            "text" : raw_data_each.xpath(f"(//span[@class='f3 push-content'])[{no_comment}]/text()")[0],
            "time" : raw_data_each.xpath(f"(//span[@class='push-ipdatetime'])[{no_comment}]/text()")[0]
        })
    
    ## 當該輪迴圈在進行完上面的迴圈所有留言爬取後，將討論項目分頁的連接、發佈者、發佈內容、發佈時間、所有留言資訊
    ## 儲存至以 Dict 的形式儲存至 data_to_df 中
    data_to_df.append({
        "link" : base_link+link,
        "writer" : raw_data_each.xpath("(//span[@class='article-meta-value'])[1]/text()")[0],
        "title" : raw_data_each.xpath("(//span[@class='article-meta-value'])[3]/text()")[0],
        "time" :  raw_data_each.xpath("(//span[@class='article-meta-value'])[4]/text()")[0],
        "comments" : comments ## 因此每輪的迴圈將覆蓋上一次迴圈的值
    })

## STEP 4: 匯出爬蟲結果

In [28]:
## 使用 pandas 套件，將儲存在 data_to_df 爬取的所有討論項目與其內容代入 DataFrame function，
## 轉化成 Table 的形式後即可透過匯出語法成果匯出資料
df = pd.DataFrame(data_to_df)
## 將爬取的內容以 csv（excel 格式）匯出
df.to_csv("20211201_Web_Scrapy_Final_Data.csv",encoding='utf-8')

In [29]:
## 查看前 5 行的資料
df.head()

,link,writer,title,time,comments
0,https://www.ptt.cc/bbs/LAW/M.1639278563.A.F42....,nextsolar (昨天的月亮),[問題] 債務人行蹤不行 法院要如何拘提呢？,Sun Dec 12 11:09:21 2021,"[{'author': 'maniaque', 'text': ': 條文有寫啊,執達員執行..."
1,https://www.ptt.cc/bbs/LAW/M.1639292679.A.A1C....,opasdaddsdd (bluesky),[問題] 工安意外死亡賠償,Sun Dec 12 15:04:37 2021,"[{'author': 'maniaque', 'text': ': 只要是合法車輛,就算沒..."
2,https://www.ptt.cc/bbs/LAW/M.1639313357.A.B30....,obillyo (綠島),[問題] 關於不動產設定抵押權問題,Sun Dec 12 20:49:15 2021,"[{'author': 'obillyo', 'text': ': 更正錯字：銀行為第一順位..."
3,https://www.ptt.cc/bbs/LAW/M.1639355860.A.0F9....,szg0417 (藍莓燒),[問題] 三審有必要再寫書狀嗎!?,Mon Dec 13 08:37:38 2021,"[{'author': 'stevenchiang', 'text': ': 三審法律審，只..."
4,https://www.ptt.cc/bbs/LAW/M.1639384334.A.868....,"bcs (= =""frailty..gggg XD)",[問題]冷氣商或冷氣業務發存證,Mon Dec 13 16:32:11 2021,"[{'author': 'KKyosuke', 'text': ': 先再問一次原因 問完..."
